In [7]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

url = '/Users/amirrezakamkar/Desktop/Works_in_progress/Ironhack/5_5th_week/Project/2nd_project/data/clean/combined_data_test.txt'
df = pd.read_csv(url)

In [66]:
df.head()

,client_id,visitor_id,visit_id,process_step,date_time,clnt_tenure_yr,clnt_tenure_mnth,clnt_age,gendr,num_accts,bal,calls_6_mnth,logons_6_mnth,Variation,next_step,is_step_back
176639,555,402506806_56087378777,637149525_38041617439_716659,start,2017-04-15 12:57:56,3.0,46.0,29.5,U,2.0,25454.66,2.0,6.0,Test,step_1,False
176638,555,402506806_56087378777,637149525_38041617439_716659,step_1,2017-04-15 12:58:03,3.0,46.0,29.5,U,2.0,25454.66,2.0,6.0,Test,step_2,False
176637,555,402506806_56087378777,637149525_38041617439_716659,step_2,2017-04-15 12:58:35,3.0,46.0,29.5,U,2.0,25454.66,2.0,6.0,Test,step_3,False
176636,555,402506806_56087378777,637149525_38041617439_716659,step_3,2017-04-15 13:00:14,3.0,46.0,29.5,U,2.0,25454.66,2.0,6.0,Test,confirm,False
176640,555,402506806_56087378777,637149525_38041617439_716659,confirm,2017-04-15 13:00:34,3.0,46.0,29.5,U,2.0,25454.66,2.0,6.0,Test,NaN,False


In [87]:
# Ensure date_time is in datetime format
df['date_time'] = pd.to_datetime(df['date_time'])

# Sort by client_id, visit_id, and date_time to ensure chronological order
df = df.sort_values(by=['client_id', 'visit_id', 'date_time'])

# Define the process steps in the correct order
process_order = ['start', 'step_1', 'step_2', 'step_3', 'confirm']

# Create a new column that shows the next step
df['next_step'] = df.groupby(['client_id', 'visit_id'])['process_step'].shift(-1)

# Identify step-back errors
df['is_step_back'] = df.apply(
    lambda row: process_order.index(row['next_step']) < process_order.index(row['process_step']) 
    if pd.notnull(row['next_step']) else False, axis=1
)

# Calculate total steps and total errors
total_steps = len(df)  # Total number of rows (steps)
total_step_back_errors = df['is_step_back'].sum()  # Sum of step-back errors

# Calculate the overall error rate
total_error_rate = total_step_back_errors / total_steps * 100

print(f"Total number of steps: {total_steps}")
print(f"Total number of step-back errors: {total_step_back_errors}")
print(f"Total error rate: {total_error_rate:.6f}%")



Total number of steps: 176641
Total number of step-back errors: 16248
Total error rate: 9.198317%


In [46]:
# Total number of visitors (unique visit_id)
total_visitors = df['visit_id'].nunique()

# Number of visitors who reached the 'confirm' step
visitors_with_confirm = df[df['process_step'] == 'confirm']['visit_id'].nunique()

# Calculate the completion rate
completion_rate = (visitors_with_confirm / total_visitors) * 100

print(f"Total number of visitors: {total_visitors}")
print(f"Number of visitors who reached the 'confirm' step: {visitors_with_confirm}")
print(f"Completion rate: {completion_rate:.2f}%")


Total number of visitors: 37122
Number of visitors who reached the 'confirm' step: 21725
Completion rate: 58.52%


In [79]:
test_average_age = df['clnt_age'].mean()
print ("The average age of the control group is", test_average_age)

The average age of the control group is 48.72221624651129


In [83]:
test_average_history = df['clnt_tenure_yr'].mean()
print ("The average history of the control group is", test_average_history, "years")

The average history of the control group is 12.182024558284883 years
